In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

In [ ]:
df = pd.read_csv('Ulst6Jan87.csv') #historic gravity data
HistGravData = gpd.GeoDataFrame(df[['id', 'OBS (mgal)']], #Keep only the ID and the reading in milligals.
                            geometry=gpd.points_from_xy(df['LONG'], df['LAT']),crs='epsg:4326') 

In [ ]:
HistGravData.head() #sanity check

In [ ]:
HistGravData.to_crs("EPSG:4326")

In [ ]:
HistGravData.to_file('Hist.shp',crs='EPSG:4326') #specified crs as by default it was creating .shp with source CRS

In [ ]:
Hist1 = gpd.read_file('Hist.shp')

In [ ]:
df = pd.read_csv('OSNIAbsoluteGravity.csv') #Absolute Gravity Data
AbsGravData = gpd.GeoDataFrame(df['Station Name'], #Keep only Station Name
                            geometry=gpd.points_from_xy(df['LONG'], df['LAT']),crs='epsg:4326') 

In [ ]:
AbsGravData.head()

In [ ]:
AbsGravData.to_file('AbData.shp',crs='EPSG:3857') #Will throw UserWarning about column name length, this is OK - **future dev work**

In [ ]:
AbData = gpd.read_file('AbData.shp')

In [ ]:
ABbuffer = AbData.buffer(0.1)

In [ ]:
ABbuffer.to_file('AB_buffer.shp',crs='EPSG:4326') #specified crs as by default it was creating .shp with source CRS

In [ ]:
AB_buffer = gpd.read_file('AB_buffer.shp')
AB_buffer.to_crs('EPSG:4326')

In [ ]:
AB_buffer.crs

In [ ]:
CombinedBuffer = AB_buffer.geometry.unary_union

In [ ]:
within_CombinedBuffer = Hist1[Hist1.geometry.within(CombinedBuffer)]

In [265]:
within_CombinedBuffer.head()

,id,OBS (mgal),geometry
10,U1 2A01,981531.93,POINT (-6.65120 55.20580)
11,U1 2B02,981525.02,POINT (-6.60530 55.20500)
12,U1 2B03,981521.47,POINT (-6.58750 55.20780)
13,U1 2B04,981522.03,POINT (-6.57780 55.20880)
14,U1 2B05,981522.82,POINT (-6.56980 55.21230)


In [ ]:
within_CombinedBuffer.to_file('withincombined.shp',crs='EPSG:4326')

In [ ]:
m = folium.Map (tiles="cartodb positron")
AB_buffer.explore(m=m, popup=False, fill_color = "White", legend_name = "10k buffer of Absolute Gravity Stations",)
m


In [335]:
m = AbsGravData.explore('Station Name',marker_type='marker', popup=True, legend=False, tiles= "cartodb positron")

AB_buffer.explore(m=m, popup=False, fill_color = "White", legend_name = "10k buffer of Absolute Gravity Stations",)

within_CombinedBuffer.explore(m=m, popup=True, legend=False,)

m